In [1]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
LEARNED_THR = 0.00001
IOU_THR = 0.7
PREDICTION_PATH = "dataset/predictions/11-predictions.csv"

In [3]:
PREDICTION_IMAGE_PATH = Path("dataset/images/test")

test_df = pd.read_csv(PREDICTION_PATH).rename(
    columns={"x_min": "xmin", "y_min": "ymin", "x_max": "xmax", "y_max": "ymax"}
)
test_df.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
110868,ID_UkxRz8.jpg,healthy,0.002495,0.709486,7.307233,172.228775,524.275635
98103,ID_eFBxoD.jpg,cssvd,0.008252,0.000000,578.398132,215.805939,951.877319
156347,ID_coFJm9.jpg,healthy,0.001070,0.000000,393.652649,162.612518,416.000000
243269,ID_EWolCN.jpeg,cssvd,0.000029,3673.874756,1982.941162,4032.000000,3022.166016
13662,ID_bgDyl3.JPG,healthy,0.005362,0.005751,862.137817,176.183121,1962.579834


In [4]:
class_id = {j: i for i, j in enumerate(sorted(test_df["class"].unique()))}
test_df["class_id"] = test_df["class"].map(class_id)
class_id

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [5]:
test_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,healthy,0.543476,12.096285,4.124999,4000.000000,1639.684082,2
1,ID_cWEAQI.jpeg,healthy,0.044651,290.501282,1408.081909,1155.421631,1788.485474,2
2,ID_cWEAQI.jpeg,healthy,0.036566,272.056458,0.000000,1397.510864,449.755157,2
3,ID_cWEAQI.jpeg,anthracnose,0.021089,1302.949341,824.583374,2883.873779,1613.518188,0
4,ID_cWEAQI.jpeg,healthy,0.015316,16.070602,83.582870,2406.208252,1603.114502,2


In [6]:
import torch

def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}


converted_labels = list(PREDICTION_IMAGE_PATH.glob("*"))
converted_labels = [i.name for i in converted_labels]

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	print(len(preds))
	print(list(preds.keys())[:10])
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels}

In [7]:
test_df.sample(2)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
236985,ID_MAbZNE.jpg,anthracnose,0.000029,423.871307,0.630432,710.953613,67.064026,0
57843,ID_xQCzG9.jpg,cssvd,0.000168,0.119004,124.447662,254.193207,906.999573,1


In [8]:
predictions = get_preds_data(test_df, thr=None)

len(predictions)

1626
['ID_A16nzu.jpg', 'ID_A1Euyz.jpg', 'ID_A1HcV0.jpeg', 'ID_A4ZdJC.jpeg', 'ID_A5SFUW.jpeg', 'ID_A6Fogi.jpeg', 'ID_ABDCyn.jpeg', 'ID_ACg6Qf.jpeg', 'ID_AFi8A1.jpg', 'ID_AHlc9P.jpg']


1626

In [9]:
# predictions["ID_CGnAYP.jpg"]

In [10]:
predictions["ID_CGnAYP.jpg"].keys()

dict_keys(['boxes', 'scores', 'labels'])

In [11]:
from torchvision.ops import nms

def apply_nms(predictions: dict[str, torch.Tensor], iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression (NMS) to reduce overlapping boxes.

    Args:
        predictions (dict): Dictionary containing keys ['boxes', 'scores', 'labels'].
        iou_threshold (float): Intersection over Union (IoU) threshold for NMS.

    Returns:
        dict: Filtered predictions after applying NMS.
    """
    filtered_predictions = {}
    for image_id, data in predictions.items():
        boxes = data["boxes"]
        scores = data["scores"]
        labels = data["labels"]

        if boxes.numel() == 0:
            # If no boxes, skip this image
            filtered_predictions[image_id] = data
            continue

        # Perform NMS for each class separately
        keep_indices = []
        for label in labels.unique():
            label_mask = labels == label
            label_boxes = boxes[label_mask]
            label_scores = scores[label_mask]
            indices = nms(label_boxes, label_scores, iou_threshold)
            keep_indices.extend(label_mask.nonzero(as_tuple=True)[0][indices].tolist())

        # Filter boxes, scores, and labels based on NMS results
        keep_indices = torch.tensor(keep_indices, dtype=torch.long)
        filtered_predictions[image_id] = {
            "boxes": boxes[keep_indices],
            "scores": scores[keep_indices],
            "labels": labels[keep_indices],
        }

    return filtered_predictions

# Apply NMS to predictions
filtered_predictions = apply_nms(predictions, iou_threshold=IOU_THR)

In [12]:
# for k, v in predictions.items():
#     r = filtered_predictions[k]
#     if len(v["boxes"]) != len(r["boxes"]):
#         print(k, len(v["boxes"]), len(r["boxes"]))
#         print("Boxes before NMS:", v["boxes"])
#         print("Boxes after NMS:", r["boxes"])
#         print("Scores before NMS:", v["scores"])
#         print("Scores after NMS:", r["scores"])
#         print("Labels before NMS:", v["labels"])
#         print("Labels after NMS:", r["labels"])
#         print()
#         break

In [13]:
filtered_data = []
id_to_label = {v: k for k, v in class_id.items()}
for image_id, data in filtered_predictions.items():
    boxes = data["boxes"].tolist()
    scores = data["scores"].tolist()
    labels = data["labels"].tolist()
    for box, score, label in zip(boxes, scores, labels):
        filtered_data.append({
            "Image_ID": image_id,
            "class": id_to_label[label],
            "confidence": score,
            "ymin": box[1],
            "xmin": box[0],
            "ymax": box[3],
            "xmax": box[2],
            "class_id": label
        })

filtered_df = pd.DataFrame(filtered_data)
filtered_df.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
0,ID_cWEAQI.jpeg,anthracnose,0.021089,1302.949341,824.583374,2883.873779,1613.518188,0
1,ID_cWEAQI.jpeg,anthracnose,0.014170,772.970886,647.254883,2418.604736,1616.861206,0
2,ID_cWEAQI.jpeg,anthracnose,0.005855,1645.428467,744.967163,3189.329834,1587.856445,0
3,ID_cWEAQI.jpeg,anthracnose,0.005826,764.870300,8.801698,2300.326660,790.475098,0
4,ID_cWEAQI.jpeg,anthracnose,0.005553,896.456665,932.427612,2597.975098,1597.602539,0


In [14]:
len(filtered_df), len(test_df)

(243586, 243900)

In [15]:
filtered_df["class"].value_counts()

class
healthy        91495
cssvd          83317
anthracnose    68774
Name: count, dtype: int64

In [16]:
filtered_df["class"].value_counts(normalize=True)

class
healthy        0.375617
cssvd          0.342043
anthracnose    0.282340
Name: proportion, dtype: float64

In [17]:
files = list(PREDICTION_IMAGE_PATH.glob("*"))

In [18]:
test_df: pd.DataFrame = filtered_df.copy()

In [19]:
test_records = test_df.to_dict(orient="records")

test_records[:5]

[{'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.021089378744363785,
  'ymin': 1302.9493408203125,
  'xmin': 824.5833740234375,
  'ymax': 2883.873779296875,
  'xmax': 1613.5181884765625,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.014170008711516857,
  'ymin': 772.9708862304688,
  'xmin': 647.2548828125,
  'ymax': 2418.604736328125,
  'xmax': 1616.8612060546875,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.005854760762304068,
  'ymin': 1645.428466796875,
  'xmin': 744.9671630859375,
  'ymax': 3189.329833984375,
  'xmax': 1587.8564453125,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.005825720727443695,
  'ymin': 764.8703002929688,
  'xmin': 8.801697731018066,
  'ymax': 2300.32666015625,
  'xmax': 790.47509765625,
  'class_id': 0},
 {'Image_ID': 'ID_cWEAQI.jpeg',
  'class': 'anthracnose',
  'confidence': 0.005552630

In [20]:
file_id = [i.name for i in files]
file_id[:2]

['ID_cWEAQI.jpeg', 'ID_NtqErb.jpg']

In [21]:
file_okay = set(test_df["Image_ID"].values)
file_nokay = set(file_id) - file_okay

for f in file_nokay:
    test_records.append(
		{
			"Image_ID": f,
		}
	)

In [22]:
test_df = pd.DataFrame(test_records)

test_df.sample(10)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
86411,ID_JfmHHi.jpeg,healthy,0.000059,0.191690,2607.409668,416.230896,3024.000000,2
31351,ID_bu374h.jpg,healthy,0.000508,0.080720,271.531067,27.658695,368.768433,2
177955,ID_tvu9FP.jpg,healthy,0.000029,0.006561,0.000000,90.482864,412.995453,2
141362,ID_zgeF64.jpg,healthy,0.000022,1944.174805,724.781738,2043.114258,989.582031,2
66404,ID_L9vR9c.jpeg,healthy,0.000357,43.374382,0.000000,2447.706055,382.753265,2
225729,ID_XkOWhr.jpg,healthy,0.000037,350.160767,576.483521,473.925079,706.552551,2
210995,ID_FmDuR4.jpeg,cssvd,0.000208,1235.845825,300.777435,1280.000000,517.844971,1
160550,ID_F5v3g7.JPG,healthy,0.000079,3095.697510,1346.247192,3264.000000,2320.577393,2
5679,ID_rtVHRR.jpg,cssvd,0.000016,9.163094,656.826904,248.110123,911.141357,1
172727,ID_OwHyW3.jpg,anthracnose,0.000044,1949.741211,1081.470947,2046.866211,1534.252197,0


In [23]:
test_df["class"].value_counts()

class
healthy        91495
cssvd          83317
anthracnose    68774
Name: count, dtype: int64

In [24]:
test_df.to_csv(PREDICTION_PATH.replace("-prediction", "-submission"), index=False)

In [25]:
PREDICTION_PATH.replace("-prediction", "-submission")

'dataset/predictions/11-submissions.csv'